In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors
import seaborn as sns

In [ ]:
# Load the dataset
data = pd.read_csv('pima_cleaned.csv')

# Separate features and target
X = data.drop('Outcome', axis=1)  # Features
y = data['Outcome']  # Target (for reference, not used in clustering)